In [ ]:
import os

import utils.data_util as du
import utils.file_util as fu

from utils.Config import Config


import numpy as np
import random

import copy


import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import wfdb
from random import shuffle
import random
from utils.data_util import denoise
import scipy.stats as st

import utils.file_util as fu

import pandas as pd
import matplotlib.pyplot as plt
import wfdb
from random import shuffle
from ecgdetectors import Detectors

os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [ ]:
def get_data(config):
    ludb_data_dir = Config.LUDB_RECORD_DIR
    postfix = 'ii'
    fs = 250
    wave_len = config.wave_len
    wave_half = int(wave_len/2)
    data_type = config.data_type
    baseline = config.baseline
    normalize = config.normalize
    FILENAME = config.fname_data
    FILTER = config.filtered
    print(FILENAME)
    
    record_list = du.get_ids(ludb_data_dir)
    data = {}
    labels = {}
    true_labels = {}
    cnt = 0
    r_ref = []
    r_ans = []
    R_errs = []
    
    for number,pid in enumerate(record_list):
        print('{}/{},{}'.format(number+1,len(record_list),pid))
        R_all = []
        record_path  = os.path.join(ludb_data_dir,pid)
        record = wfdb.rdrecord(record_path)
        lead_II_idx = np.where(np.array(record.sig_name)==postfix)[0][0]
        record = record.p_signal[:,lead_II_idx]
        record = record.flatten()

        record = denoise(record, fs*2)
        record =np.array(record)
   
        record = down_sample(record, times = 2)
  
        
        atr = wfdb.rdann(os.path.join(ludb_data_dir,pid),extension=postfix)


        atr_symbols = np.array(atr.symbol)
        atr_points = np.array(np.array(atr.sample)/2, dtype=np.int)
        
        r_list = np.where(np.array(atr_symbols)=='N')[0]
        r_list = atr_points[r_list]
        if len(r_list)==0:
            continue
        thr_ = 0.15 
        fs_ = 250
        
        detectors = Detectors(fs_)
        r_peaks = detectors.pan_tompkins_detector(record)
      
        r_ref.append(np.array(r_list))
        r_ans.append(np.array(r_peaks))

        TOL = 37.5
        start = -1
        end = -1
        for k in range(len(r_peaks)):
            if np.abs(r_peaks[k]-r_list[0]) <= TOL:
                start = k
            if np.abs(r_peaks[k]-r_list[-1]) <= TOL:
                end = k + 1


        per_data = []
        per_label = []
        for j in range(start, end):
            r_idx = r_peaks[j]
            if r_idx - wave_half < 0 or r_idx+wave_half >= len(record):
                continue
            lead_seg = record[r_idx - wave_half: r_idx + wave_half]
            include_R_symbols_idx = atr_points[(atr_points>=r_idx-TOL)&(atr_points<r_idx + TOL)]

            R_seg_symbols = atr_symbols[np.isin(atr_points,include_R_symbols_idx)]
            index = -1
            tmp_cnt =0
            for k in range(len(R_seg_symbols)):
                if R_seg_symbols[k] == 'N':
                    tmp_cnt +=1
                    index = k

            if index != -1:               
                true_r_idx =  include_R_symbols_idx[index]
                R_all.append(true_r_idx)
                R_errs.append(r_idx- true_r_idx)



                include_symbols_idx = atr_points[(atr_points>=true_r_idx - wave_half)&(atr_points<true_r_idx + wave_half)]
                seg_symbols = atr_symbols[np.isin(atr_points,include_symbols_idx)]
                index = int(np.where(include_symbols_idx==true_r_idx)[0][0])
                if index >=4 and index+5 <= len(seg_symbols):
                    if (seg_symbols[index-4:index+5] == ['(', 'p', ')', '(', 'N', ')', '(', 't', ')']).all():    
                        p_start, p_end = include_symbols_idx[index-4]- r_idx + wave_half, include_symbols_idx[index-2]- r_idx + wave_half
                        r_start, r_end = include_symbols_idx[index-1]- r_idx + wave_half, include_symbols_idx[index+1]- r_idx + wave_half
                        t_start, t_end = include_symbols_idx[index+3]- r_idx + wave_half, include_symbols_idx[index+4]- r_idx + wave_half
                        label = [p_start, p_end, r_start, r_end, t_start, t_end]
                        per_data.append(lead_seg)
                        per_label.append(label)
                        cnt+=1
                

        print('record_beat_cnt:', len(per_data))
        data[pid] = per_data
        labels[pid] =  per_label
    
  


    import utils.file_util as fu
    file_name = FILENAME
    fu.save((data, labels), file_name)
    print('success') 
    return data,labels


In [ ]:

config  = Config()
config.dataset = 'ludb'
config.refresh()
get_data(config)
